In [95]:
!ls 

README.md                          meilleur_proposition_pour_oppo.sql
analysis.ipynb                     myapi.py
data_sources                       reflexion.ipynb
env                                requirements.txt


In [96]:
pip install pandas 


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [97]:
pip install tabulate


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [98]:
pip install duckdb


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [99]:
import pandas as pd
from typing import Dict
from dataclasses import dataclass
from tabulate import tabulate
import duckdb

In [100]:
df_oportunity = pd.read_csv("data_sources/data_samples/opportunity_test.csv")

In [101]:
df_oportunity.head(3)

,Id,A_une_proposition_de_sa_banque__c,Age_emprunteur__c,Appel_Automatique__c,Avancement__c,BanquePrincipaleEmp__c,Connaissances_en_immobilier__c,CreatedDate,Deja_souscrit_credit_immo__c,DurSouhaitePret__c,...,TechMail_ContratDeTravailCoEmprunteur__c,TechMail_ContratDeTravailEmprunteur__c,TotalProposition__c,TotCharges__c,TotRev__c,Tx_Completude_Pieces__c,TxEndetApres__c,TypBien__c,TypProj__c,UsagBien__c
0,0065q00000AlAK0AAN,0,33.0,0,2.0,CIC,2.0,2023-10-20 04:44:34,1,20.0,...,CDI période d'essai terminée,CDI période d'essai terminée,1.0,1750.0,7731.67,0.0,41.75,Autres,Acquisition+travaux,Investissement locatif
1,0065q00000AlAK5AAN,0,30.0,0,4.0,BOURSORAMA BANQUE,1.0,2023-10-20 04:44:34,0,20.0,...,NaN,CDI période d'essai terminée,NaN,0.0,1695.00,0.0,33.86,Appartement,Acquisition,Résidence principale
2,0065q00000AlAKPAA3,0,50.0,1,4.0,BNP PARIBAS,2.0,2023-10-20 04:56:59,1,20.0,...,NaN,CDD,4.0,500.0,25500.00,0.0,12.00,Maison,Acquisition,Investissement locatif


## Partie 1 : Analyse

### Question 1 : Déterminer le profil des personnes ayant soumis des opportunités (les demandeurs de prêt immobilier)

En considérant uniquement les champs suivant : Age_emprunteur__c', 'BanquePrincipaleEmp__c', 'Deja_souscrit_credit_immo__c', 'TechMail_CategorieProfessionnelleEmpru__c', 'TechMail_ContratDeTravailEmprunteur__c' , on peut déterminer le profil des demandeurs en calculant certains indicateurs. On pourra aussi effectuer des analyses croisées.

In [102]:

@dataclass
class AnalyseDeProfil:
    """Classe pour analyser le profil des emprunteurs"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.clean_data()
    
    def clean_data(self):
        """Nettoie les données"""

        '''
            age_min = -940.0 est une valeur aberrante. Cela montre qu'il y'a un probleme au niveau des données.
            On peut effectuer un nétoyage des données suivi d'une validation. Il en est de meme pour age_max = 144.
            Donc il faut éliminer les valeurs aberrante ou fixer un interval d'age valable et ainsi eliminer 
            l'ensemble des lignes que ne matche pas.
        '''
        
        # Suppression des valeurs aberrantes pour l'âge
        self.df = self.df[self.df['Age_emprunteur__c'].between(18, 100)]
    
    def age_emprunteur_statistics(self) -> Dict:
        """Calcule les statistiques sur l'âge"""
        age_emprunteur_stats = {
            'moyenne': self.df['Age_emprunteur__c'].mean(),
            'mediane': self.df['Age_emprunteur__c'].median(),
            'min': self.df['Age_emprunteur__c'].min(),
            'max': self.df['Age_emprunteur__c'].max(),
            
        }
        
        return age_emprunteur_stats
    
    def banque_principale_analysis(self) -> Dict:
        """Analyse la répartition des banques principales"""
        return {
            'repartition': self.df['BanquePrincipaleEmp__c'].value_counts().to_dict(),
            'pourcentage': (self.df['BanquePrincipaleEmp__c'].value_counts(normalize=True) * 100).to_dict(),
            'nombre_total': len(self.df['BanquePrincipaleEmp__c'].unique())
        }
    
    def credit_immo_precedent(self) -> Dict:
        """Analyse de l'expérience précédente en crédit immobilier"""
        return {
            'repartition': self.df['Deja_souscrit_credit_immo__c'].value_counts().to_dict(),
            'pourcentage': (self.df['Deja_souscrit_credit_immo__c'].value_counts(normalize=True) * 100).to_dict()
        }
    
    def categorie_professionnelle_analysis(self) -> Dict:
        """Analyse sur les catégories professionnelles"""
        return {
            'repartition': self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts().to_dict(),
            'pourcentage': (self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts(normalize=True) * 100).to_dict(),
            'top_5': self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts().head().to_dict()
        }
    
    def type_contract_travail_analysis(self) -> Dict:
        """Analyse les types de contrats de travail"""
        return {
            'repartition': self.df['TechMail_ContratDeTravailEmprunteur__c'].value_counts().to_dict(),
            'pourcentage': (self.df['TechMail_ContratDeTravailEmprunteur__c'].value_counts(normalize=True) * 100).to_dict()
        }
    
    
    
    def generer_rapport(self) -> Dict:
        """Génère un rapport d'analyse"""
        return {
            'statistiques_age': self.age_emprunteur_statistics(),
            'analyse_banques': self.banque_principale_analysis(),
            'analyse_experience_credit': self.credit_immo_precedent(),
            'analyse_categories_pro': self.categorie_professionnelle_analysis(),
            'analyse_contrats': self.type_contract_travail_analysis(),
        }

def main():
    # Chargement des données 
    df = pd.read_csv("data_sources/data_samples/opportunity_test.csv")
    df_oportunity_ = df[['Age_emprunteur__c', 'BanquePrincipaleEmp__c', 
                                'Deja_souscrit_credit_immo__c', 
                                'TechMail_CategorieProfessionnelleEmpru__c',
                                'TechMail_ContratDeTravailEmprunteur__c'
                                ]]
    
    # Création de l'analyseur
    analyzer = AnalyseDeProfil(df_oportunity_)
    
    # Génération du rapport
    rapport = analyzer.generer_rapport()
    
    # Affichage 
    print("\n=== Profil Type des Emprunteurs ===\n")
    
    # Age
    print("1. Profil d'âge:")
    print(f"- Âge moyen: {rapport['statistiques_age']['moyenne']:.1f} ans")
    print(f"- Âge médian: {rapport['statistiques_age']['mediane']:.1f} ans")
    
    # Catégorie professionnelle
    print("\n2. Top 5 des catégories professionnelles:")
    for cat, count in rapport['analyse_categories_pro']['top_5'].items():
        pct = rapport['analyse_categories_pro']['pourcentage'][cat]
        print(f"- {cat}: {count} personnes ({pct:.1f}%)")
    
    # Expérience crédit
    print("\n3. Expérience en crédit immobilier:")
    for exp, pct in rapport['analyse_experience_credit']['pourcentage'].items():
        print(f"- {exp}: {pct:.1f}%")
    
    # Type de contrat
    print("\n4. Types de contrat de travail:")
    for contract, pct in rapport['analyse_contrats']['pourcentage'].items():
        print(f"- {contract}: {pct:.1f}%")

if __name__ == "__main__":
    main()


=== Profil Type des Emprunteurs ===

1. Profil d'âge:
- Âge moyen: 37.3 ans
- Âge médian: 36.0 ans

2. Top 5 des catégories professionnelles:
- Salarié du privé: 125596 personnes (72.2%)
- Salarié du public: 27028 personnes (15.5%)
- Dirigeant de société: 4828 personnes (2.8%)
- Retraité: 4155 personnes (2.4%)
- Travailleur indépendant: 2801 personnes (1.6%)

3. Expérience en crédit immobilier:
- 0: 59.2%
- 1: 40.8%

4. Types de contrat de travail:
- CDI période d'essai terminée: 79.1%
- Fonctionnaire ou assimilé: 14.3%
- Éducation nationale: 2.3%
- CDD: 1.9%
- CDI période d'essai non terminée: 1.4%
- Intérimaire: 0.6%
- Autres: 0.3%
- Multi-employeurs: 0.0%
- Intermittent du spectacle: 0.0%


### Question 2 : Utiliser SQL

Pour chaque opportunité, sélectionnez la proposition de prêt la plus avantageuse parmi toutes les propositions disponibles.

`Une opportunité peut avoir plusieurs propositions de prêt de la part de différentes banques, avec des conditions telles que le taux d’intérêt, la durée et le taux d’assurance.`

In [103]:
df_proposition = pd.read_csv("data_sources/data_samples/propositions_test.csv")
df_proposition.head(2)

# Opportunity__c : ID de la proposition
# Partenaire__c : Référence de la Banque
# TXHA__c : Taux d'intéret

,Id,CreatedDate,Opportunity__c,Partenaire__c,TXHA__c,DureePret_Mois__c,TauxAss__c,Etape_Source__c
0,a0PIV000009qPjV2AU,2024-01-01 00:45:52,006IV00000Exs17YAB,0012000000YI8d8AAD,4.70,240.0,0.42,99-Non éligible à la transmission / Source: SI...
1,a0PIV000009qPjW2AU,2024-01-01 00:45:52,006IV00000Exs17YAB,0012000000YI8d8AAD,4.85,300.0,0.42,99-Non éligible à la transmission / Source: SI...


In [104]:
df_banques = pd.read_csv("data_sources/data_samples/banques_test.csv")
df_banques.head(5)

# Nom de la banque

,Id,CreatedDate,Name
0,0011i000002MVpNAAW,2018-10-05 10:50:32,CFCAL
1,0011i000002MVu7AAG,2018-10-05 10:59:31,CFCAL
2,0011i000002MWCHAA4,2018-10-05 11:24:29,CFCAL
3,0011i000003PFlRAAW,2018-10-29 15:18:49,CREDIT DU NORD NORD DE FRANCE
4,0011i000003PFtzAAG,2018-10-29 15:30:39,CREDIT DU NORD NORD DE FRANCE


In [105]:
df_opportunity = pd.read_csv("data_sources/data_samples/opportunity_test.csv")
df_opportunity.head(2)

,Id,A_une_proposition_de_sa_banque__c,Age_emprunteur__c,Appel_Automatique__c,Avancement__c,BanquePrincipaleEmp__c,Connaissances_en_immobilier__c,CreatedDate,Deja_souscrit_credit_immo__c,DurSouhaitePret__c,...,TechMail_ContratDeTravailCoEmprunteur__c,TechMail_ContratDeTravailEmprunteur__c,TotalProposition__c,TotCharges__c,TotRev__c,Tx_Completude_Pieces__c,TxEndetApres__c,TypBien__c,TypProj__c,UsagBien__c
0,0065q00000AlAK0AAN,0,33.0,0,2.0,CIC,2.0,2023-10-20 04:44:34,1,20.0,...,CDI période d'essai terminée,CDI période d'essai terminée,1.0,1750.0,7731.67,0.0,41.75,Autres,Acquisition+travaux,Investissement locatif
1,0065q00000AlAK5AAN,0,30.0,0,4.0,BOURSORAMA BANQUE,1.0,2023-10-20 04:44:34,0,20.0,...,NaN,CDI période d'essai terminée,NaN,0.0,1695.00,0.0,33.86,Appartement,Acquisition,Résidence principale


In [106]:
# Créer une connexion DuckDB
con = duckdb.connect()

query = """ SELECT opp.Id, pro.Opportunity__c,
            pro.Partenaire__c , pro.TXHA__c, pro.DureePret_Mois__c , ban.Name
            FROM read_csv_auto('data_sources/data_samples/propositions_test.csv') pro
            JOIN read_csv_auto('data_sources/data_samples/opportunity_test.csv') opp 
                ON pro.Opportunity__c = opp.Id 
            JOIN read_csv_auto('data_sources/data_samples/banques_test.csv') ban 
                ON pro.Partenaire__c = ban.Id
                ORDER BY pro.TXHA__c, pro.DureePret_Mois__c
    """

# Exécuter et convertit en DataFrame
result = con.execute(query).df()

print("\nResultat apres jointure (sans filtre)")
print(result)

con.close()




Meilleures propositions par opportunité:
                       Id      Opportunity__c       Partenaire__c  TXHA__c  \
0      006IV00000FMf44YAD  006IV00000FMf44YAD  001D000001PJDfaIAH      0.0   
1      006IV00000FMf44YAD  006IV00000FMf44YAD  0011i000004G5cQAAS      0.0   
2      006IV00000FPmGzYAL  006IV00000FPmGzYAL  0011i000004G5dOAAS      0.0   
3      006IV00000FVGCjYAP  006IV00000FVGCjYAP  001D000001uriUQIAY      0.0   
4      006IV00000F3hFxYAJ  006IV00000F3hFxYAJ  001D000001urolnIAA      0.0   
...                   ...                 ...                 ...      ...   
51921  006IV00000F86lxYAB  006IV00000F86lxYAB  001D0000029OvhBIAS      NaN   
51922  006IV00000DnNFJYA3  006IV00000DnNFJYA3  0012000000WmV09AAF      NaN   
51923  006IV00000EFA8fYAH  006IV00000EFA8fYAH  0012000000WmV28AAF      NaN   
51924  006IV00000DAqFcYAL  006IV00000DAqFcYAL  001D0000029OvhDIAS      NaN   
51925  006IV00000FGiFmYAL  006IV00000FGiFmYAL  001D0000029OvhDIAS      NaN   

       DureePret_Mois

In [107]:
# Créer une connexion DuckDB
con = duckdb.connect()

# Requête SQL
query = """
    WITH RankedProposals AS (
        SELECT 
            o.Id,
            lp.Partenaire__c,
            lp.Opportunity__c,
            lp.TXHA__c,
            lp.DureePret_Mois__c,
            ba.Name,
            ROW_NUMBER() OVER (
                PARTITION BY o.Id 
                ORDER BY lp.TXHA__c, lp.DureePret_Mois__c
            ) as rank
        FROM read_csv_auto('data_sources/data_samples/opportunity_test.csv') o
        JOIN read_csv_auto('data_sources/data_samples/propositions_test.csv') lp 
            ON o.Id = lp.Opportunity__c
        JOIN read_csv_auto('data_sources/data_samples/banques_test.csv') ba
            ON lp.Partenaire__c = ba.Id
    )
    SELECT 
        Opportunity__c as "ID de l'opportunité",
        Partenaire__c as "ID de la proposition",
        TXHA__c as "Taux d'intérêt",
        DureePret_Mois__c as "Durée du prêt",
        Name as "Nom de la banque"
    FROM RankedProposals
    WHERE rank = 1
    ORDER BY Opportunity__c;
    """

result = con.execute(query).df()

print("\nMeilleures propositions par opportunité:")
print(tabulate(result.head(10), headers='keys', tablefmt='grid'))

con.close()
   


Meilleures propositions par opportunité:
+----+-----------------------+------------------------+------------------+-----------------+---------------------------------------------------------------------------+
|    | ID de l'opportunité   | ID de la proposition   |   Taux d'intérêt |   Durée du prêt | Nom de la banque                                                          |
+====+=======================+========================+==================+=================+===========================================================================+
|  0 | 0065q00000AlCtOAAV    | 0012000000WmV96AAF     |             4.3  |             300 | CAISSE D'EPARGNE EVREUX PETITE CITE                                       |
+----+-----------------------+------------------------+------------------+-----------------+---------------------------------------------------------------------------+
|  1 | 0065q00000AlDP3AAN    | 001D0000029OvhXIAS     |             0    |             240 | LA BANQUE POSTALE MA